In [11]:
%pip install pymongo


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [17]:
from pymongo import MongoClient

def insert_data():
    client = MongoClient('mongodb://localhost:27017/')
    db = client.local  # 'local' 데이터베이스 사용

    # 책 데이터 삽입
    books = [
        {"title": "Kafka on the Shore", "author": "Haruki Murakami", "year": 2002},
        {"title": "Norwegian Wood", "author": "Haruki Murakami", "year": 1987},
        {"title": "1Q84", "author": "Haruki Murakami", "year": 2009}
    ]
    db.books.insert_many(books)

    # 영화 데이터 삽입
    movies = [
        {"title": "Inception", "director": "Christopher Nolan", "year": 2010, "rating": 8.8},
        {"title": "Interstellar", "director": "Christopher Nolan", "year": 2014, "rating": 8.6},
        {"title": "The Dark Knight", "director": "Christopher Nolan", "year": 2008, "rating": 9.0}
    ]
    db.movies.insert_many(movies)

    # 사용자 행동 데이터 삽입
    user_actions = [
        {"user_id": 1, "action": "click", "timestamp": "2023-04-12T08:00:00Z"},
        {"user_id": 1, "action": "view", "timestamp": "2023-04-12T09:00:00Z"},
        {"user_id": 2, "action": "purchase", "timestamp": "2023-04-12T10:00:00Z"}
    ]
    db.user_actions.insert_many(user_actions)

    print("Data inserted successfully")
    client.close()

if __name__ == "__main__":
    insert_data()



Data inserted successfully


In [19]:
# 문제1: 특정 장르의 책 찾기
client = MongoClient('mongodb://localhost:27017/')
db = client.local  # 'local' 데이터베이스 사용
collection = db.books

query = {"genre":"fantagy"}

for doc in collection.find(query):
    print(doc)

In [21]:
# 문제2: 감독별 평균 영화 평점 계산
movies_collection = db.movies
query = [{"$group":{"_id":"$director", "avg_rating":{"$avg": "$rating"}}}]

for data in movies_collection.aggregate(query):
    print(data)

{'_id': 'Christopher Nolan', 'avg_rating': 8.799999999999999}


In [22]:
#문제3: 특정 사용자의 최근 행동 조회
user_actions_collection = db.user_actions
query = {"user_id":1}
sort_criteria = [("timestamp", -1)]

actions = user_actions_collection.find(query).sort(sort_criteria).limit(5)

for data in actions:
    print(data)


{'_id': ObjectId('6628b7c2ac85060f521d0057'), 'user_id': 1, 'action': 'view', 'timestamp': '2023-04-12T09:00:00Z'}
{'_id': ObjectId('6628b8f5ac85060f521d0063'), 'user_id': 1, 'action': 'view', 'timestamp': '2023-04-12T09:00:00Z'}
{'_id': ObjectId('6628adc2ac85060f521d0041'), 'user_id': 1, 'action': 'view', 'timestamp': '2023-04-12T09:00:00Z'}
{'_id': ObjectId('6628b74fac85060f521d004d'), 'user_id': 1, 'action': 'view', 'timestamp': '2023-04-12T09:00:00Z'}
{'_id': ObjectId('6628b74fac85060f521d004c'), 'user_id': 1, 'action': 'click', 'timestamp': '2023-04-12T08:00:00Z'}


In [23]:
#문제4: 출판 연도별 책의 수 계산
books_collection = db.books

query = [{"$group": {"_id":"$year", "count":{"$sum":1}}},{"$sort": {"count":-1}}]

for data in books_collection.aggregate(query):
    print(data)

{'_id': 1987, 'count': 4}
{'_id': 2009, 'count': 4}
{'_id': 2002, 'count': 4}


In [56]:
#문제5: 특정 사용자의 행동 유형 업데이트
from datetime import datetime

def update_user_date(user_id, date, old_action, new_action):
    user_actions_collection = db.user_actions
    query = {"user_id":user_id, "action": old_action, "timestamp": {"$lt": date}}
    update = {"$set": {"action": new_action}}
    result = user_actions_collection.update_many(query, update)
    print(f"Updated {result.modified_count} document")

update_user_date(1, datetime(2023, 4, 10), "view", "seen" )
 
    

Updated 0 document
